In [65]:
import xmltodict
import pandas as pd
import copy
import csv
import os
import langid
import multiprocessing
BASE_DIR = '/home/antonio/lattes_base/Collection'

In [2]:
def get_author(doc):
    author = {}
    valid = 0
    try :
        if(doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['DOUTORADO']['@NOME-CURSO']  == 'Ciências da Computação'):
            author_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS']
            author_id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
            author_complete_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO']

            author['author_id'] = author_id
            author['author_name'] = author_complete_name
            author['author_ref'] = author_name
            artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
            for artigo in artigos:
                lang, log_prob = langid.classify(artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'])
                if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês" and lang  == 'en'):
                    valid = 1
                    author['paper'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
                    author['paper_issn'] = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
                    author['paper_ano'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
                    author['paper_home_page'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']       
                    string = ''
                    for autores in artigo['AUTORES']:     
                        string += 'ordem: ' + autores['@ORDEM-DE-AUTORIA'] + '\n'
                        string += 'name: ' + autores['@NOME-COMPLETO-DO-AUTOR'] + '\n'
                        string += 'name_ref ' + autores['@NOME-PARA-CITACAO']  + '\n'
                    author['coauthors'] = string
    except:
        pass
    if valid == 1:
        return author
    return None

In [3]:
def worker(files, return_num, return_dict):
    authors = []
    for file in files:
        with open(file + '/curriculo.xml', encoding="ISO-8859-1") as fd:
            doc = xmltodict.parse(fd.read())
        authors.append(get_author(doc))
    return_dict[return_num] = authors

### get all files

In [4]:
files_list = []

for folder in os.listdir(BASE_DIR):
    path = os.path.join(BASE_DIR, folder)
    if(os.path.isdir(path) == True and not folder.startswith('.')):
        for files in os.listdir(path):
            if not files.startswith('.'):
                file = os.path.join(path, files)
                files_list.append(file)


### Define tasks

In [ ]:
n_threads = 4
n_data = len(files_list)
if(n_data%n_threads == 0):
        division = int(n_data/n_threads)
else:
    division = int(n_data/n_threads)+1
    
manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []
for i in range(n_threads):
    init = i*division
    end = (i+1)*division
    p = multiprocessing.Process(target=worker, args=(files_list[init:end], i, return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

### Generate dataframe and save

In [63]:
values = []
for value in return_dict.values():
    for item in value:
        if item:
            values.append(item)

In [66]:
df = pd.DataFrame(values)

In [70]:
df.shape

(457, 8)

In [68]:
df.to_csv('data.csv', sep='|')

In [ ]:
'''author = {}
author_inf = {}

author_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS']
author_id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
author_complete_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO']

author_inf['id'] = author_id
author_inf['name'] = author_complete_name
author_inf['ref_name'] = author_name
author['informacoes'] = author_inf

artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
pub = []
for artigo in artigos:
    if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
        paper = {}
        paper['titulo'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
        paper['issn'] = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
        paper['ano'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
        paper['home_page'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']       
        
        
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'])
        #print(artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN'])
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO'])
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'])
        #authors = []
        #authors_inf = {}
        #for autores in artigo['AUTORES']:            
        #    authors_inf['nome'] = autores['@NOME-COMPLETO-DO-AUTOR']
        #    authors_inf['name_ref']  = autores['@NOME-PARA-CITACAO']
        #    authors_inf['ordem'] = autores['@ORDEM-DE-AUTORIA']
            #authors[autores['@NOME-PARA-CITACAO']] = autors_inf.copy()
        #    authors.append(copy.deepcopy(authors_inf))
            
            #print(autores['@NOME-COMPLETO-DO-AUTOR'])
            #print(autores['@NOME-PARA-CITACAO'])
            #print(autores['@ORDEM-DE-AUTORIA'])
            
        #paper['autores'] = copy.deepcopy(authors)
        pub.append(copy.deepcopy(paper))
        print('\n\n')
author['artigos'] = pub[:]'''